# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [93]:
%matplotlib widget

In [94]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np
import calendar

def plot_assignment() :
    # Prepare df(s) for plotting
    df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
    '''Line graph of the record high and record low temperatures by day of the year over the period 2005-2014.'''
    # split the Date column into year, day of year, and month
    df['Year'] = df['Date'].str[0:4]
    df['DayOfYear'] = df['Date'].str[5:]
    df['Month'] = df['DayOfYear'].str[:2]
    # divide Data_Value by 10 for neater decimal
    df['Temp'] = df['Data_Value']/10
    # filter out day of year '02-29'
    df = df[df['DayOfYear']!='02-29']

    # split df for 2015 and 2004-2014
    df1 = df[df['Year']!='2015'].sort_values(by='Date')
    df2 = df[df['Year']=='2015'].sort_values(by='Date')

    # generate a df or list with the max/min temp per day of the year .groupby
    mins = df1[df1['Element'] == 'TMIN']
    dfmin = mins.groupby(by='DayOfYear').min('Temp').reset_index()

    maxs = df1[df1['Element'] == 'TMAX']
    dfmax = maxs.groupby(by='DayOfYear').max('Temp').reset_index()

    # start plot figure
    plt.figure()

    # plot the min values per date in blue circles
    dates = dfmin['DayOfYear'].tolist()
    minlist = dfmin['Temp'].tolist()

    plt.plot(dates, minlist, '#00e4ff')

    # plot the max values per date in red circles
    dates = dfmax['DayOfYear'].tolist()
    maxlist = dfmax['Temp'].tolist()

    plt.plot(dates, maxlist, '#ff9408')

    '''The area between the record high and record low temperatures for each day should be shaded.'''
    # eg. plt.fill_between(range(len(linear_data)), linear_data, exponential_data, facecolor='red', alpha=0.11))
    plt.fill_between(dates, minlist, maxlist, facecolor='#448ee4', alpha=0.11)

    '''Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.'''
    # df of min/max 2015 data are with <2014 min/max
    mins = df2[df2['Element'] == 'TMIN']
    df2015min = mins.groupby(by='DayOfYear').min('Temp').reset_index()
    list2015min = df2015min['Temp'].tolist()
    maxs = df2[df2['Element'] == 'TMAX']
    df2015max = maxs.groupby(by='DayOfYear').max('Temp').reset_index()
    list2015max = df2015max['Temp'].tolist()
    dates = df2015min['DayOfYear'].tolist()
    df2015 = pd.DataFrame({'DayOfYear' : dates, 'min2015' : list2015min, 'max2015' : list2015max, 'min' : minlist, 'max' : maxlist})

    record_highs = df2015[df2015['max2015'] > df2015['max']]
    plotdates = record_highs['DayOfYear'].tolist()
    plottemps = record_highs['max2015'].tolist()

    plt.plot(plotdates, plottemps, 'rd', alpha=.5)

    record_lows = df2015[df2015['min2015'] < df2015['min']]
    plotdates = record_lows['DayOfYear'].tolist()
    plottemps = record_lows['min2015'].tolist()

    plt.plot(plotdates, plottemps, 'bd', alpha=.5)
    # scatter plot() 2015 highs and lows depending on logic above
    # Pretty-up the figure

    plt.title('Ann Arbor, MI\nHigh and Low Temperatures\n(2005-2014)')
    plt.tick_params(labelrotation=.45)
    plt.xlabel('Day of the Year')
    plt.ylabel('Temperature ($^\circ$C)')
    plt.tick_params(axis='both', which='both', bottom=False, left=False)
    plt.grid(b=True, axis='y', alpha=.1)
    plt.legend(['Lows','Highs','Record High (2015)', 'Record Low (2015)'])
    plt.box(False)
    plt.xticks([1,32,60,91,121,152,182,213,244,274,305,335], calendar.month_name[1:13], rotation=80)
    # adjust the subplot so the text doesn't run off the image
    plt.subplots_adjust(bottom=0.25)
    return plt.show()

plot_assignment()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …